# CDP Agentkit 工具包

`CDP Agentkit` 工具包包含使 LLM Agent 能够与 [Coinbase Developer Platform](https://docs.cdp.coinbase.com/) 交互的工具。该工具包提供了一个 CDP SDK 的包装器，允许 Agent 执行链上操作，如转账、交易和智能合约交互。

## 概述

### 集成详情

| 类 | 包 | 可序列化 | JS 支持 | 包最新 |
| :--- | :--- | :---: | :---: | :---: |
| CdpToolkit | `cdp-langchain` | ❌ | ❌ |  ![PyPI - Version](https://img.shields.io/pypi/v/cdp-langchain?style=flat-square&label=%20) |

### 工具功能

该工具包提供以下工具：

1. **get_wallet_details** - 获取 MPC 钱包的详细信息
2. **get_balance** - 获取特定资产的余额
3. **request_faucet_funds** - 从水龙头请求测试代币
4. **transfer** - 在地址之间转账资产
5. **trade** - 交易资产（仅限主网）
6. **deploy_token** - 部署 ERC-20 代币合约
7. **mint_nft** - 从现有合约铸造 NFT
8. **deploy_nft** - 部署新的 NFT 合约
9. **register_basename** - 为钱包注册一个基本名称

我们鼓励您添加自己的工具，包括 CDP 和 web2 API，以创建满足您需求的定制 Agent。

## 设置

总体来说，我们将执行以下操作：

1. 安装 langchain 包
2. 设置您的 CDP API 凭证
3. 初始化 CDP 包装器和工具包
4. 使用 `toolkit.get_tools()` 将工具传递给您的 Agent

要启用对单个工具的自动追踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

此工具包位于 `cdp-langchain` 包中：

In [ ]:
%pip install -qU cdp-langchain

#### 设置环境变量

要使用此工具包，您必须先设置以下环境变量，以访问[CDP API](https://docs.cdp.coinbase.com/mpc-wallet/docs/quickstart) 来创建钱包和进行链上交互。您可以在[CDP Portal](https://cdp.coinbase.com/)上免费注册 API 密钥：

In [ ]:
import getpass
import os

for env_var in [
    "CDP_API_KEY_NAME",
    "CDP_API_KEY_PRIVATE_KEY",
]:
    if not os.getenv(env_var):
        os.environ[env_var] = getpass.getpass(f"Enter your {env_var}: ")

# Optional: Set network (defaults to base-sepolia)
os.environ["NETWORK_ID"] = "base-sepolia"  # or "base-mainnet"

## 实例化

现在我们可以实例化我们的工具集：

In [ ]:
from cdp_langchain.agent_toolkits import CdpToolkit
from cdp_langchain.utils import CdpAgentkitWrapper

# Initialize CDP wrapper
cdp = CdpAgentkitWrapper()

# Create toolkit from wrapper
toolkit = CdpToolkit.from_cdp_agentkit_wrapper(cdp)

## 工具

查看[可用工具](#tool-features)：

In [ ]:
tools = toolkit.get_tools()
for tool in tools:
    print(tool.name)

## 在代理中使用

我们需要一个 LLM 或聊天模型：

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

初始化代理，使用以下工具：

In [ ]:
from langgraph.prebuilt import create_react_agent

tools = toolkit.get_tools()
agent_executor = create_react_agent(llm, tools)

示例用法：

In [ ]:
example_query = "Send 0.005 ETH to john2879.base.eth"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

已将 0.005 eth 转账至 john2879.base.eth。
转账交易哈希：0x78c7c2878659a0de216d0764fc87eff0d38b47f3315fa02ba493a83d8e782d1e
转账交易链接：https://sepolia.basescan.org/tx/0x78c7c2878659a0de216d0764fc87eff0d38b47f3315fa02ba493a83d8e782d1

## CDP 工具包特定功能

### 钱包管理

该工具包维护一个 MPC 钱包。钱包数据可以导出和导入，以便在会话之间持久化：

In [ ]:
# Export wallet data
wallet_data = cdp.export_wallet()

# Import wallet data
values = {"cdp_wallet_data": wallet_data}
cdp = CdpAgentkitWrapper(**values)

### 网络支持

该工具包支持[多个网络](https://docs.cdp.coinbase.com/cdp-sdk/docs/networks)

### 无Gas费交易

部分操作支持在Base主网上进行无Gas费交易：
- USDC 转账
- EURC 转账
- cbBTC 转账

## API 参考

有关所有 CDP 功能和配置的详细文档，请访问 [CDP 文档](https://docs.cdp.coinbase.com/mpc-wallet/docs/welcome)。